In [1]:
from sklearn.datasets import load_breast_cancer
breast_cancer_data = load_breast_cancer()


In [2]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# Load the Breast Cancer dataset
dataset = load_breast_cancer()
features = pd.DataFrame(dataset.data, columns=dataset.feature_names)
labels = pd.Series(dataset.target)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Save the dataframes to CSV files
X_train.to_csv('train_features.csv', index=False)
X_test.to_csv('test_features.csv', index=False)
y_train.to_csv('train_labels.csv', index=False)
y_test.to_csv('test_labels.csv', index=False)


In [3]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Define the feature selection
feature_selector = SelectKBest(score_func=f_classif, k=10)  # Select top 10 features
selected_features = feature_selector.fit_transform(X_train, y_train)

# Print the selected features
selected_columns = X_train.columns[feature_selector.get_support()]
print("Selected Features:", selected_columns)


Selected Features: Index(['mean radius', 'mean perimeter', 'mean area', 'mean concavity',
       'mean concave points', 'worst radius', 'worst perimeter', 'worst area',
       'worst concavity', 'worst concave points'],
      dtype='object')


In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Define the ANN model
ann_model = MLPClassifier(max_iter=1000)

# Define the parameter grid for Grid Search
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001]
}

# Set up Grid Search with Cross-Validation
grid_search_cv = GridSearchCV(estimator=ann_model, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search_results = grid_search_cv.fit(X_train, y_train)

# Print the best parameters found by Grid Search
print(f"Optimal Parameters: {grid_search_results.best_params_}")


Optimal Parameters: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (50, 50), 'solver': 'adam'}


In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Retrieve the best parameters from Grid Search
best_params = grid_search_results.best_params_
best_ann_model = MLPClassifier(**best_params, max_iter=1000)
best_ann_model.fit(X_train, y_train)

# Predict and evaluate the model
predictions = best_ann_model.predict(X_test)
print(f"Accuracy on Test Data: {accuracy_score(y_test, predictions):.4f}")
print("Classification Report:\n", classification_report(y_test, predictions))


Accuracy on Test Data: 0.9708
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.92      0.96        63
           1       0.96      1.00      0.98       108

    accuracy                           0.97       171
   macro avg       0.98      0.96      0.97       171
weighted avg       0.97      0.97      0.97       171



In [7]:
!pip install streamlit


  Using cached altair-5.3.0-py3-none-any.whl.metadata (9.2 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached watchdog-4.0.1-py3-none-win_amd64.whl.metadata (37 kB)
  Using cached toolz-0.12.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.7 MB 1.7 MB/s eta 0:00:06
    -----------


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import streamlit as st
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

st.title("Breast Cancer Prediction App")

# Load preprocessed data
X_train = pd.read_csv('train_features.csv')
X_test = pd.read_csv('test_features.csv')
y_train = pd.read_csv('train_labels.csv')
y_test = pd.read_csv('test_labels.csv')

# Initialize and train the ANN model
trained_model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001, max_iter=1000)
trained_model.fit(X_train, y_train)

# User input
st.sidebar.header("Enter Features")
user_input = st.sidebar.text_input("Enter feature values separated by commas (e.g., 0.1,0.2,0.3,...):")

if user_input:
    user_data = pd.DataFrame([user_input.split(",")], columns=X_train.columns)
    prediction = trained_model.predict(user_data)
    st.write(f"Prediction: {'Malignant' if prediction[0] == 1 else 'Benign'}")

# Model evaluation
y_test_pred = trained_model.predict(X_test)
st.write(f"Accuracy on Test Data: {accuracy_score(y_test, y_test_pred):.4f}")


c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
